In [1]:
import pandas as pd
import numpy as np
import pickle
from typing import Optional, Union, Tuple
from cmapPy.pandasGEXpress.parse import parse
import numpy as np
import json
from collections import Counter
import sys
from numpy.core.multiarray import ndarray
import sys
import csv 
import requests 
import xml.etree.ElementTree as ET 
from xml.dom import minidom

/Users/dweepa/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Parsing the XML file

## Importing drugbank dataset

In [2]:
drugbank = pd.read_csv("drugbank.csv",low_memory=False)
drugbank = drugbank.dropna(how='all', axis='columns')
drugbank.head(2)

,Unnamed: 0,atc-level1,atc-level2,atc-level3,atc-level4,atc-level5,inchi,inchi_key,name,pubchem,...,syn59,syn6,syn60,syn61,syn62,syn63,syn64,syn7,syn8,syn9
0,0,B,B01,B01A,B01AE,B01AE02,NaN,NaN,Lepirudin,46507011.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,L,L01,L01X,L01XC,L01XC06,NaN,NaN,Cetuximab,46507042.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
p = pd.read_csv("../Data/GSE70138_Broad_LINCS_pert_info.txt",delimiter="\t")
p.canonical_smiles = p.canonical_smiles.str.upper()
p.head(2)

,pert_id,canonical_smiles,inchi_key,pert_iname,pert_type
0,BRD-K70792160,CCN(CC)CCCCN1C2CCCCC2OC2CCC(CL)CC12,GYBXAGDWMCJZJK-UHFFFAOYSA-N,10-DEBC,trt_cp
1,BRD-K68552125,CCCCCCCCCCCCCC(=O)O[C@@H]1[C@@H](C)[C@]2(O)[C@...,PHEDXBVPIONUQT-RGYGYFBISA-N,phorbol-myristate-acetate,trt_cp


In [4]:
X = pd.read_pickle("../Data/full")

In [5]:
# two different smiles for the same inchi_key
print(drugbank[drugbank.inchi_key=='QHMWCHQXCUNUAK-UHFFFAOYSA-N'].smiles.tolist()[0])
print(p[p.inchi_key=='QHMWCHQXCUNUAK-UHFFFAOYSA-N'].canonical_smiles.tolist()[0])

FC1=CC=C(C=C1)C(N1C=CN=C1)(C1=CC=CC=C1)C1=CC=CC=C1F
FC1CCC(CC1)C(C1CCCCC1)(C1CCCCC1F)N1CCNC1


## Get the atc codes via drugbank

## Import the final dataset

In [8]:
# pd.DataFrame(li).to_csv("atc-codes.csv")
atc = pd.read_csv("atc-codes.csv")
atc.head(2)

,Unnamed: 0,inchi_key,level1,level2,level3,level4,level5,name,smiles
0,0,IETKPTYAGKZLKY-UHFFFAOYSA-N,NaN,NaN,NaN,NaN,NaN,Balaglitazone,CN1C(COC2CCC(CC3SC(=O)NC3=O)CC2)NC2CCCCC2C1=O
1,1,QHMWCHQXCUNUAK-UHFFFAOYSA-N,G,G01,G01A,G01AF,G01AF18,Flutrimazole,FC1CCC(CC1)C(C1CCCCC1)(C1CCCCC1F)N1CCNC1


In [9]:
atc.isnull().sum()

Unnamed: 0       0
inchi_key        0
level1        1484
level2        1484
level3        1484
level4        1484
level5        1484
name          1069
smiles           0
dtype: int64

## Import original ATC standalone dataset

In [10]:
atc_data = pd.read_excel("atc.xlsx")
atc_data.head(2)

,Class ID,Preferred Label,Synonyms,Definitions,Obsolete,CUI,Semantic Types,Parents,ATC LEVEL,Is Drug Class,Semantic type UMLS property
0,http://purl.bioontology.org/ontology/UATC/A03AX13,silicones,NaN,NaN,False,C0037114,http://purl.bioontology.org/ontology/STY/T109|...,http://purl.bioontology.org/ontology/UATC/A03AX,5.0,NaN,http://purl.bioontology.org/ontology/STY/T109|...
1,http://purl.bioontology.org/ontology/UATC/J01DB07,cefatrizine,NaN,NaN,False,C0007545,http://purl.bioontology.org/ontology/STY/T195|...,http://purl.bioontology.org/ontology/UATC/J01DB,5.0,NaN,http://purl.bioontology.org/ontology/STY/T195|...


## Get pert_iname from pert_id

In [81]:
id_to_name = dict()
name_to_id = dict()
tar = np.unique(X.target)
for i in range(len(tar)):
    a = p[p.pert_id==tar[i]].pert_iname
    if(a.to_string().split()[1]==''):
        a = a.to_string().split()[0]
    else:
        a = a.to_string().split()[1]
    a=a.translate({ord(')'): None})
    a=a.translate({ord(']'): None})
#     sys.stdout.write('\r%d/%d\t\t%s' % (i, len(tar),a))
    print('%d/%d\t\t%s' % (i, len(tar),a))
    id_to_name[tar[i]]=a
    name_to_id[a]=tar[i]

0/2170		balaglitazone
1/2170		flutrimazole
2/2170		KBG
3/2170		biperiden
4/2170		drospirenone
5/2170		bupivacaine
6/2170		garcinol
7/2170		naftopidil
8/2170		nitrendipine
9/2170		ethinyl-estradiol
10/2170		meptazinol
11/2170		lofexidine
12/2170		alizapride
13/2170		mepivacaine
14/2170		fulvestrant
15/2170		XL-888
16/2170		metoprolol
17/2170		maraviroc
18/2170		chlorphenamine
19/2170		bupropion
20/2170		lisofylline
21/2170		BAY-K8644
22/2170		telmesteine
23/2170		docetaxel
24/2170		CPI-613
25/2170		fusidic-acid
26/2170		esmolol
27/2170		labetalol
28/2170		bopindolol
29/2170		isofloxythepin
30/2170		kakonein
31/2170		galeterone
32/2170		racecadotril
33/2170		doxycycline
34/2170		norethindrone
35/2170		flecainide
36/2170		verapamil
37/2170		argatroban
38/2170		tipranavir
39/2170		propranolol
40/2170		GSK-429286A
41/2170		imiloxan
42/2170		granisetron
43/2170		carvedilol
44/2170		pravastatin
45/2170		beclomethasone-dipropionate
46/2170		lorazepam
47/2170		cefepime
48/2170		nutlin-3
49/2170

413/2170		JNJ-38877605
414/2170		amoxapine
415/2170		cefoperazone
416/2170		epitiostanol
417/2170		RG-2833
418/2170		caffeine
419/2170		S-18986
420/2170		celecoxib
421/2170		GSK-1059615
422/2170		minaprine
423/2170		clofoctol
424/2170		dasabuvir
425/2170		BMS-299897
426/2170		SU-11274
427/2170		lamivudine
428/2170		PF-477736
429/2170		levosimendan
430/2170		roflumilast
431/2170		lersivirine
432/2170		ecabet
433/2170		roquinimex
434/2170		cobimetinib
435/2170		azacitidine
436/2170		foretinib
437/2170		ixabepilone
438/2170		rottlerin
439/2170		penitrem-a
440/2170		raclopride
441/2170		nepafenac
442/2170		oxybuprocaine
443/2170		favipiravir
444/2170		AGI-6780
445/2170		VE-822
446/2170		clemizole
447/2170		SAR-245408
448/2170		ciprofloxacin
449/2170		GSK-1904529A
450/2170		LDN-193189
451/2170		AKT-inhibitor-1-2
452/2170		nebivolol
453/2170		LY-2334737
454/2170		PD-184352
455/2170		luteolin
456/2170		nicotine
457/2170		ledipasvir
458/2170		dronedarone
459/2170		raltegravir
460/2170		dicloxa

855/2170		glasdegib
856/2170		trimethobenzamide
857/2170		levalbuterol
858/2170		PD-123319
859/2170		glimepiride
860/2170		adoprazine
861/2170		theobromine
862/2170		enprofylline
863/2170		taranabant
864/2170		levonorgestrel
865/2170		ciclesonide
866/2170		dacarbazine
867/2170		yohimbine
868/2170		danegaptide
869/2170		SR-27897
870/2170		gimeracil
871/2170		rotundine
872/2170		liranaftate
873/2170		niclosamide
874/2170		asymmetrical-dimethylarginine
875/2170		esculin
876/2170		tofacitinib
877/2170		NF-449
878/2170		I-BET-151
879/2170		dolutegravir
880/2170		bindarit
881/2170		tamibarotene
882/2170		omecamtiv-mecarbil
883/2170		setiptiline
884/2170		GSK-1070916
885/2170		hydroflumethiazide
886/2170		glibenclamide
887/2170		tonabersat
888/2170		trientine
889/2170		clozapine
890/2170		efaproxiral
891/2170		conivaptan
892/2170		idebenone
893/2170		zardaverine
894/2170		GSK-256066
895/2170		AZD-8330
896/2170		doxepin
897/2170		ABT-724
898/2170		SB-525334
899/2170		PF-03758309
900/2170		etop

1300/2170		cilostazol
1301/2170		altretamine
1302/2170		ORE-1001
1303/2170		milrinone
1304/2170		phenylbutyrate
1305/2170		apixaban
1306/2170		acetylcysteine
1307/2170		picotamide
1308/2170		ZK-200775
1309/2170		GDC-0879
1310/2170		CI-966
1311/2170		haloperidol
1312/2170		adatanserin
1313/2170		pevonedistat
1314/2170		AR-A014418
1315/2170		PI-103
1316/2170		avasimibe
1317/2170		lisinopril
1318/2170		MK-2206
1319/2170		palmitoylethanolamide
1320/2170		CGS-20625
1321/2170		terbinafine
1322/2170		epothilone-b
1323/2170		torin-2
1324/2170		ALW-II-38-3
1325/2170		etizolam
1326/2170		lacosamide
1327/2170		tyrphostin-AG-1478
1328/2170		amsacrine
1329/2170		ZK-93426
1330/2170		KIN001-055
1331/2170		ENMD-2076
1332/2170		SB-743921
1333/2170		BRD-K68548958
1334/2170		phorbol-myristate-acetate
1335/2170		lidamidine
1336/2170		PF-03814735
1337/2170		FR-180204
1338/2170		quetiapine
1339/2170		flumatinib
1340/2170		GDC-0152
1341/2170		golvatinib
1342/2170		sertindole
1343/2170		diacerein
1344/2170		l

1749/2170		chlorpropamide
1750/2170		PD-173074
1751/2170		theophylline
1752/2170		malotilate
1753/2170		EMD-1214063
1754/2170		rupatadine
1755/2170		SB-239063
1756/2170		GSK-J4
1757/2170		IOX2
1758/2170		salirasib
1759/2170		honokiol
1760/2170		clonidine
1761/2170		AZD-8931
1762/2170		bepotastine
1763/2170		baicalin
1764/2170		flumazenil
1765/2170		AZD-5363
1766/2170		PS-178990
1767/2170		felbamate
1768/2170		AZD-2014
1769/2170		QL-XII-47
1770/2170		quinine
1771/2170		ftorafur
1772/2170		lopinavir
1773/2170		genz-644282
1774/2170		goserelin-acetate
1775/2170		PF-562271
1776/2170		tanaproget
1777/2170		motesanib
1778/2170		linifanib
1779/2170		hexachlorophene
1780/2170		bosutinib
1781/2170		reboxetine
1782/2170		verteporfin
1783/2170		nemonapride
1784/2170		levcromakalim
1785/2170		cevimeline
1786/2170		saracatinib
1787/2170		XMD11-85H
1788/2170		GSK-1059615
1789/2170		THZ-2-98-01
1790/2170		HG-6-64-01
1791/2170		WH-4-025
1792/2170		WZ-4-145
1793/2170		XMD-16144
1794/2170		BI-2536
1795/

In [83]:
print(len(name_to_id))
len(id_to_name)

1826


2170

## Get ATC codes from OG data

In [108]:
# Unnamed: 0	inchi_key	level1	level2	level3	level4	level5	name	smiles
li1=[]
count=0
for i in name_to_id.keys():
    if(i in list(atc_data['Preferred Label'])):\
        count+=1
    atcdict = dict()
    try:
        atc_class = atc_data[atc_data['Preferred Label']==i]['Class ID'].tolist()[0].split('/')[-1]
        print("%s\t%s\t%s"%(i , name_to_id[i], atc_class))
        atcdict["name"] = i
        atcdict["pert_id"] = name_to_id[i]
        atcdict["level1"]=atc_class[0]
        atcdict["level2"]=atc_class[0:2]
        atcdict["level3"]=atc_class[0:3]
        atcdict["level4"]=atc_class[0:5]
        atcdict["level5"]=atc_class
        atcdict['inchi_key']=''
        atcdict['smiles']=''
        li1.append(atcdict)
    except Exception as e:
        pass

flutrimazole	BRD-A00218260	D01AC16
biperiden	BRD-A00546892	N04AA02
drospirenone	BRD-A00938334	G03AC10
bupivacaine	BRD-A01636364	N01BB01
nitrendipine	BRD-A02006392	C08CA08
meptazinol	BRD-A02710418	N02AX05
lofexidine	BRD-A02990301	N07BC04
alizapride	BRD-A03061970	A03FA05
mepivacaine	BRD-A03216249	N01BB03
fulvestrant	BRD-A03249105	L02BA03
metoprolol	BRD-A03623303	C07AB02
maraviroc	BRD-A04352665	J05AX09
chlorphenamine	BRD-A04553218	R06AB04
bupropion	BRD-A05186015	N06AX12
docetaxel	BRD-A05821830	L01CD02
esmolol	BRD-A07395371	C07AB09
labetalol	BRD-A07440155	C07AG01
bopindolol	BRD-A07563059	C07AA17
racecadotril	BRD-A08187463	A07XA04
doxycycline	BRD-A08545410	A01AB22
flecainide	BRD-A09472452	C01BC04
verapamil	BRD-A09533288	C08DA01
argatroban	BRD-A10012892	B01AE03
tipranavir	BRD-A10039652	J05AE09
propranolol	BRD-A10070317	C07AA05
granisetron	BRD-A10967948	A04AA02
carvedilol	BRD-A10977446	C07AG02
pravastatin	BRD-K31447894	C10AA03
lorazepam	BRD-A11990600	N05BA06
cefepime	BRD-A12077521	J01DE01
cis

clioquinol	BRD-K09255212	P01AA02
levofloxacin	BRD-K09471561	J01MA12
mibefradil	BRD-K09549677	C08CX01
dabrafenib	BRD-K09951645	L01XE23
sulfasalazine	BRD-K10670311	A07EC01
phenylbutazone	BRD-K10843433	M02AA01
tolcapone	BRD-K10852020	N04BX01
latanoprost	BRD-K10961822	S01EE01
gemfibrozil	BRD-K11129031	C10AB04
norfloxacin	BRD-K11196887	S01AE02
alectinib	BRD-K11267252	L01XE36
phenformin	BRD-K11399644	A10BA01
benzbromarone	BRD-K11717138	M04AB03
glipizide	BRD-K12219985	A10BB07
trametinib	BRD-K12343256	L01XE25
griseofulvin	BRD-K12423485	D01BA01
sulbutiamine	BRD-K12539416	A11DA02
valdecoxib	BRD-K12994359	M01AH03
rimonabant	BRD-K13296708	A08AX01
methazolamide	BRD-K13356952	S01EC05
midostaurin	BRD-K13646352	L01XE39
vardenafil	BRD-K13926615	G04BE09
cinoxacin	BRD-K14704277	J01MB06
bemegride	BRD-K14993104	R07AB05
emedastine	BRD-K15010214	S01GX06
gemcitabine	BRD-K15108141	L01BC05
carfilzomib	BRD-K15179879	L01XX45
penfluridol	BRD-K15409150	N05AG03
nevirapine	BRD-K15502390	J05AG01
papaverine	BRD-K155671

cariprazine	BRD-K57764956	N05AX15
vilazodone	BRD-K58010567	N06AX24
rasagiline	BRD-K58114536	N04BD02
roxithromycin	BRD-K58211978	J01FA06
dexfenfluramine	BRD-K58626373	A08AA04
chlorprothixene	BRD-K59058766	N05AF03
naproxen	BRD-K59197931	G02CC02
propentofylline	BRD-K59273480	N06BC02
voglibose	BRD-K59331299	A10BF03
linopirdine	BRD-K59332007	N06BX09
tedizolid	BRD-K59436580	J01XX11
maribavir	BRD-K59524082	J05AX10
quinidine	BRD-K59632282	C01BA01
pasireotide	BRD-K59715032	H01CB05
irbesartan	BRD-K60038276	C09CA04
tiagabine	BRD-K60160658	N03AG06
paricalcitol	BRD-K60585088	H05BX02
idelalisib	BRD-K60866521	L01XX47
loperamide	BRD-K61250553	A07DA03
romidepsin	BRD-K61397605	L01XX39
thiomersal	BRD-K61443650	D08AK06
emtricitabine	BRD-K61452026	J05AF09
anagrelide	BRD-K62200014	L01XX35
dapsone	BRD-K62363391	J04BA02
venetoclax	BRD-K62391742	L01XX52
guanethidine	BRD-K62676709	C02CC02
triptorelin	BRD-K62685538	L02AE04
gabapentin	BRD-K62737565	N03AX12
lidoflazine	BRD-K62996583	C08EX01
epinephrine	BRD-K631261

In [109]:
# 1 Instance of dict
li1[0]

{'inchi_key': '',
 'level1': 'D',
 'level2': 'D0',
 'level3': 'D01',
 'level4': 'D01AC',
 'level5': 'D01AC16',
 'name': 'flutrimazole',
 'pert_id': 'BRD-A00218260',
 'smiles': ''}

In [112]:
# atc = atc.drop(['Unnamed: 0'],axis=1)
atc['pert_id'] = ''
atc.isnull().sum()

inchi_key       0
level1       1484
level2       1484
level3       1484
level4       1484
level5       1484
name         1069
smiles          0
pert_id         0
dtype: int64

In [111]:
newatc = pd.DataFrame(li1)
newatc.isnull().sum()

inchi_key    0
level1       0
level2       0
level3       0
level4       0
level5       0
name         0
pert_id      0
smiles       0
dtype: int64

# Append Data

In [117]:
wholeatc = newatc.append(atc)
wholeatc.isnull().sum()

inchi_key       0
level1       1484
level2       1484
level3       1484
level4       1484
level5       1484
name         1069
pert_id         0
smiles          0
dtype: int64

In [142]:
for i in Counter(wholeatc.name):
    if(Counter(wholeatc.name)[i]>1):
        print(i,Counter(wholeatc.name)[i])
len(wholeatc.name)

nan 1069
Saracatinib 2


2938

In [121]:
len(atc.name) + len(newatc.name)

2938

In [131]:
print(atc[atc.name=='Saracatinib'])
print('\n',newatc[newatc.name=='Saracatinib'],'\n')
print(wholeatc[wholeatc.name=='Saracatinib'])

                        inchi_key level1 level2 level3 level4 level5  \
651   OUKYUETWWIPKQR-UHFFFAOYSA-N    NaN    NaN    NaN    NaN    NaN   
1786  OUKYUETWWIPKQR-UHFFFAOYSA-N    NaN    NaN    NaN    NaN    NaN   

             name                                             smiles pert_id  
651   Saracatinib  CN1CCN(CCOC2CC(OC3CCOCC3)C3C(NC4C5OCOC5CCC4CL)...          
1786  Saracatinib  CN1CCN(CC1)CCOC2=CC(=C3C(=C2)N=CN=C3NC4=C(C=CC...          

 Empty DataFrame
Columns: [inchi_key, level1, level2, level3, level4, level5, name, pert_id, smiles]
Index: [] 

                        inchi_key level1 level2 level3 level4 level5  \
651   OUKYUETWWIPKQR-UHFFFAOYSA-N    NaN    NaN    NaN    NaN    NaN   
1786  OUKYUETWWIPKQR-UHFFFAOYSA-N    NaN    NaN    NaN    NaN    NaN   

             name pert_id                                             smiles  
651   Saracatinib          CN1CCN(CCOC2CC(OC3CCOCC3)C3C(NC4C5OCOC5CCC4CL)...  
1786  Saracatinib          CN1CCN(CC1)CCOC2=CC(=C3C(=C2)N

In [134]:
def common_member(a, b): 
    a_set = set(a) 
    b_set = set(b) 
    if (a_set & b_set): 
        print(a_set & b_set) 
    else: 
        print("No common elements")  

common_member(atc.name,newatc.name)
common_member([1,2,3],[2,3,4,5])

No common elements
{2, 3}
